In [21]:
import numpy as np
import pandas as pd
import ipynb.fs.full.data_preprocessing_after_imputing as datapre
from sklearn.metrics import confusion_matrix
import ipynb.fs.full.Random_forest as rf
## change your path for data file here
pathTrain="C:/Users/12783/Downloads/Data_impute/KNN_impute.csv" ##training
pathVali="C:/Users/12783/Downloads/Data_impute/KNN_impute_validation.csv" ## testing
pathTest="C:/Users/12783/Downloads/Data_impute/KNN_impute_test.csv" ## testing

In [22]:
def deletion(dataframe):
    data=dataframe.copy()
    data_cc=data.dropna()
    data_cc_reind=data_cc.set_index([pd.Index(range(len(data_cc)))])
    return(data_cc_reind)

In [23]:
def upsample(data):
    from sklearn.utils import resample
    df_majority=data[data.Default_ind==0]
    df_minority=data[data.Default_ind==1]
    largenumber=len(df_majority)
    df_minority_upsampled=resample(df_minority,replace=True,n_samples=int(1.2*largenumber),random_state=123)
    df_upsampled=pd.concat([pd.DataFrame(df_majority),pd.DataFrame(df_minority_upsampled)])
    return df_upsampled

KNN

In [29]:
dataTrain=pd.read_csv(pathTrain)
dataVali=pd.read_csv(pathVali)
dataTest=pd.read_csv(pathTest)

In [30]:
dataTrainup=upsample(datapre.Cate_to_object_excp_inq(dataTrain))
dataValiup=upsample(datapre.Cate_to_object_excp_inq(dataVali))
dataTest=datapre.Cate_to_object_excp_inq(dataTest)

In [27]:
parameters=[{'max_depth':list(range(1,20)),
            'criterion':["gini", "entropy"],
            'min_samples_split':[100,190,200,210,500,1000],
            'n_estimators':list(range(10,101,10))+[200],
            'min_samples_leaf':[10]+list(range(50,251,50)),
            'bootstrap':[False],
            'warm_start':[True,False],
            'max_samples':[1500,2500,2900,3000,3500,4500]},
            {'bootstrap': [True],
             'criterion': ["gini", "entropy"],
             'max_depth': list(range(1,20)),
             'max_samples': [1500,2500,2900,3000,3500,4500],
             'min_samples_leaf': [10]+list(range(50,251,50)),
             'min_samples_split': [100,190,200,210,500,1000],
             'n_estimators': list(range(10,101,10))+[200],
             'oob_score': [True,False],
             'warm_start': [True,False]}]

In [95]:
clfKNN=rf.best_Random_forest(dataTrainup, dataValiup, parameters, criteria='accuracy', randomstate=0)

In [96]:
clfKNN.best_estimator_

RandomForestClassifier(max_depth=10, max_samples=2900, min_samples_leaf=50,
                       min_samples_split=190, n_estimators=200, oob_score=True,
                       random_state=0, warm_start=True)

In [97]:
clfKNN.best_params_

{'bootstrap': True,
 'criterion': 'gini',
 'max_depth': 10,
 'max_samples': 2900,
 'min_samples_leaf': 50,
 'min_samples_split': 190,
 'n_estimators': 200,
 'oob_score': True,
 'warm_start': True}

In [80]:
frames=pd.concat([dataTrainup, dataValiup])
datafinaltrain=frames.set_index([pd.Index(range(len(frames)))])

In [81]:
rffinalknn=rf.Random_forest(datafinaltrain, nestimator=100, criter="gini", mdepth=10, misamples_split=190, 
                                 misamples_leaf=50, bootstra=True, oobscore=True, warmstart=True, maxsamples=2900, randomstate=0)

In [82]:
dataTestdu=pd.get_dummies(dataTest)
datatestx=np.array(dataTestdu.drop('Default_ind', axis=1))
datatesty=dataTestdu['Default_ind']

In [83]:
confusion_matrix(np.array(datatesty),np.array(rffinalknn.predict(datatestx)))

array([[3471, 1128],
       [  73,  328]], dtype=int64)

In [85]:
rffinalknn.score(datatestx,datatesty)

0.7598

Un mean

In [34]:
pathTrain="C:/Users/12783/Downloads/Data_impute/Unconditional_mean_impute.csv" ##training
pathVali="C:/Users/12783/Downloads/Data_impute/Unconditional_mean_impute_validation.csv" ## testing
pathTest="C:/Users/12783/Downloads/Data_impute/Unconditional_mean_impute_Test.csv" ## testing

In [35]:
dataTrain=pd.read_csv(pathTrain)
dataVali=pd.read_csv(pathVali)
dataTest=pd.read_csv(pathTest)

In [36]:
dataTrainup=upsample(datapre.Cate_to_object_excp_inq(dataTrain))
dataValiup=upsample(datapre.Cate_to_object_excp_inq(dataVali))
dataTest=datapre.Cate_to_object_excp_inq(dataTest)

In [37]:
parameters=[{'max_depth':list(range(5,15)),
            'criterion':["gini", "entropy"],
            'min_samples_split':list(range(200,801,100)),
            'bootstrap':[True],
            'oob_score':[True,False],
            'warm_start':[True,False],
            'max_samples':[500,800,1000,3000],
            'min_samples_leaf':[20,40,50,100]},
            {'max_depth':list(range(5,15)),
            'criterion':["gini", "entropy"],
            'min_samples_split':list(range(200,801,100)),
            'bootstrap':[False],
            'warm_start':[True,False],
            'max_samples':[500,800,1000,3000],
            'min_samples_leaf':[20,40,50,100]}]

In [60]:
clfunmean=rf.best_Random_forest(dataTrainup, dataValiup, parameters, criteria='accuracy', randomstate=0)

In [61]:
clfunmean.best_params_

{'bootstrap': False,
 'criterion': 'entropy',
 'max_depth': 12,
 'max_samples': 500,
 'min_samples_leaf': 40,
 'min_samples_split': 500,
 'warm_start': True}

In [38]:
frames=pd.concat([dataTrainup, dataValiup])
datafinaltrain=frames.set_index([pd.Index(range(len(frames)))])

In [93]:
rffinalunmean=rf.Random_forest(datafinaltrain, nestimator=100, criter="entropy", mdepth=12, misamples_split=500, 
                            misamples_leaf=40, bootstra=False, oobscore=False, warmstart=True, maxsamples=500, randomstate=0)

In [39]:
dataTestdu=pd.get_dummies(dataTest)
datatestx=np.array(dataTestdu.drop('Default_ind', axis=1))
datatesty=dataTestdu['Default_ind']

In [95]:
confusion_matrix(np.array(datatesty),np.array(rffinalunmean.predict(datatestx)))

array([[3768,  831],
       [  90,  311]], dtype=int64)

In [96]:
rffinalknn.score(datatestx,datatesty)

0.8158

Cart

In [45]:
pathTrain="C:/Users/12783/Downloads/Data_impute/Cart_impute.csv" ##training
pathVali="C:/Users/12783/Downloads/Data_impute/Cart_impute_Vali.csv" ## testing
pathTest="C:/Users/12783/Downloads/Data_impute/Cart_impute_Test.csv" ## testing

In [46]:
dataTrain=pd.read_csv(pathTrain)
dataVali=pd.read_csv(pathVali)
dataTest=pd.read_csv(pathTest)

In [47]:
dataTrainup=upsample(datapre.Cate_to_object_excp_inq(dataTrain))
dataValiup=upsample(datapre.Cate_to_object_excp_inq(dataVali))
dataTest=datapre.Cate_to_object_excp_inq(dataTest)

In [48]:
parameters=[{'max_depth':list(range(5,15)),
            'criterion':["gini", "entropy"],
            'min_samples_split':list(range(200,801,100)),
            'bootstrap':[True],
            'oob_score':[True,False],
            'warm_start':[True,False],
            'max_samples':[200,500,800,1000,3000],
            'min_samples_leaf':[20,40,50,100]},
            {'max_depth':list(range(5,15)),
            'criterion':["gini", "entropy"],
            'min_samples_split':list(range(200,801,100)),
            'bootstrap':[False],
            'warm_start':[True,False],
            'max_samples':[200,500,800,1000,3000],
            'min_samples_leaf':[20,40,50,100]}]

In [111]:
clfcart=rf.best_Random_forest(dataTrainup, dataValiup, parameters, criteria='accuracy', randomstate=0)

In [112]:
clfcart.best_params_

{'bootstrap': False,
 'criterion': 'gini',
 'max_depth': 10,
 'max_samples': 200,
 'min_samples_leaf': 50,
 'min_samples_split': 300,
 'warm_start': True}

In [49]:
frames=pd.concat([dataTrainup, dataValiup])
datafinaltrain=frames.set_index([pd.Index(range(len(frames)))])

In [50]:
rffinalcart=rf.Random_forest(datafinaltrain, nestimator=100, criter="gini", mdepth=10, misamples_split=300, 
                               misamples_leaf=50, bootstra=False, oobscore=False, warmstart=True, maxsamples=200, randomstate=0)

In [51]:
dataTestdu=pd.get_dummies(dataTest)
datatestx=np.array(dataTestdu.drop('Default_ind', axis=1))
datatesty=dataTestdu['Default_ind']

In [52]:
confusion_matrix(np.array(datatesty),np.array(rffinalcart.predict(datatestx)))

array([[3775,  824],
       [  99,  302]], dtype=int64)

In [53]:
rffinalcart.score(datatestx,datatesty)

0.8154

Unconditional median

In [87]:
pathTrain="C:/Users/12783/Downloads/Data_impute/Unconditional_median_impute.csv" ##training
pathVali="C:/Users/12783/Downloads/Data_impute/Unconditional_median_impute_Validation.csv" ## testing
pathTest="C:/Users/12783/Downloads/Data_impute/Unconditional_median_impute_Test.csv" ## testing

In [88]:
dataTrain=pd.read_csv(pathTrain)
dataVali=pd.read_csv(pathVali)
dataTest=pd.read_csv(pathTest)

In [89]:
dataTrainup=upsample(datapre.Cate_to_object_excp_inq(dataTrain))
dataValiup=upsample(datapre.Cate_to_object_excp_inq(dataVali))
dataTest=datapre.Cate_to_object_excp_inq(dataTest)

In [69]:
parameters=[{'max_depth':list(range(5,15)),
            'criterion':["gini", "entropy"],
            'min_samples_split':list(range(200,801,100)),
            'bootstrap':[True],
            'oob_score':[True,False],
            'warm_start':[True,False],
            'max_samples':[200,500,800,1000,2500,3000],
            'min_samples_leaf':[40,50,100]},
            {'max_depth':list(range(5,15)),
            'criterion':["gini", "entropy"],
            'min_samples_split':list(range(200,801,100)),
            'bootstrap':[False],
            'warm_start':[True,False],
            'max_samples':[200,500,800,1000,2500,3000],
            'min_samples_leaf':[40,50,100]}]

In [396]:
clfunmedian=rf.best_Random_forest(dataTrainup, dataValiup, parameters, criteria='accuracy', randomstate=0)

In [397]:
clfunmedian.best_params_

RandomForestClassifier(bootstrap=False, criterion='entropy', max_depth=12,
                       max_samples=2500, min_samples_leaf=50,
                       min_samples_split=500, random_state=0, warm_start=True)

In [90]:
frames=pd.concat([dataTrainup, dataValiup])
datafinaltrain=frames.set_index([pd.Index(range(len(frames)))])

In [91]:
rffinalunmedian=rf.Random_forest(datafinaltrain, nestimator=100, criter="entropy", mdepth=12, misamples_split=500, 
                                 misamples_leaf=40, bootstra=False, oobscore=False, warmstart=True, maxsamples=2500, randomstate=0)

In [92]:
dataTestdu=pd.get_dummies(dataTest)
datatestx=np.array(dataTestdu.drop('Default_ind', axis=1))
datatesty=dataTestdu['Default_ind']

In [93]:
confusion_matrix(np.array(datatesty),np.array(rffinalunmedian.predict(datatestx)))

array([[3783,  816],
       [  94,  307]], dtype=int64)

Listwise Deletion

In [101]:
pathTrain="C:/Users/12783/Downloads/Data_impute/Listwise_deletion.csv" ##training
pathVali="C:/Users/12783/Downloads/Data_impute/Listwise_deletion_Validation.csv" ## testing
pathTest="C:/Users/12783/Downloads/Data_impute/Listwise_deletion_Test.csv" ## testing

In [104]:
dataTrain=pd.read_csv(pathTrain)
dataVali=pd.read_csv(pathVali)
dataTest=pd.read_csv(pathTest)

In [105]:
dataTrainup=upsample(datapre.Cate_to_object_excp_inq(dataTrain))
dataValiup=upsample(datapre.Cate_to_object_excp_inq(dataVali))
dataTest=datapre.Cate_to_object_excp_inq(dataTest)

In [62]:
parameters=[{'max_depth':list(range(9,19)),
            'criterion':["gini", "entropy"],
            'min_samples_split':[10,50]+list(range(200,801,100)),
            'bootstrap':[True],
            'oob_score':[True,False],
            'warm_start':[True,False],
            'max_samples':[1000,2500,3000,3500],
            'min_samples_leaf':[1,40,50,100]},
            {'max_depth':list(range(9,19)),
            'criterion':["gini", "entropy"],
            'min_samples_split':[10,50]+list(range(200,801,100)),
            'bootstrap':[False],
            'warm_start':[True,False],
            'max_samples':[1000,2500,3000,3500],
            'min_samples_leaf':[1,40,50,100]}]


In [396]:
clfld=rf.best_Random_forest(dataTrainup, dataValiup, parameters, criteria='accuracy', randomstate=0)

In [397]:
clfld.best_params_

RandomForestClassifier(bootstrap=False, criterion='entropy', max_depth=12,
                       max_samples=2500, min_samples_leaf=50,
                       min_samples_split=500, random_state=0, warm_start=True)

In [106]:
frames=pd.concat([dataTrainup, dataValiup])
datafinaltrain=frames.set_index([pd.Index(range(len(frames)))])

In [107]:
rffinalld=rf.Random_forest(datafinaltrain, nestimator=100, criter="gini", mdepth=16, misamples_split=10, 
    misamples_leaf=1, bootstra=False, oobscore=False, warmstart=True, maxsamples=3000, randomstate=0)

In [108]:
dataTestdu=pd.get_dummies(dataTest)
datatestx=np.array(dataTestdu.drop('Default_ind', axis=1))
datatesty=dataTestdu['Default_ind']

In [109]:
confusion_matrix(np.array(datatesty),np.array(rffinalld.predict(datatestx)))

array([[3735,   75],
       [ 230,  111]], dtype=int64)

EMB

In [118]:
pathTrain="C:/Users/12783/Downloads/Data_impute/EMB_impute.csv" ##training
pathVali="C:/Users/12783/Downloads/Data_impute/EMB_impute_Vali.csv" ## testing
pathTest="C:/Users/12783/Downloads/Data_impute/EMB_impute_Test.csv" ## testing

In [119]:
dataTrain=pd.read_csv(pathTrain)
dataVali=pd.read_csv(pathVali)
dataTest=pd.read_csv(pathTest)

In [120]:
dataTrainup=upsample(datapre.Cate_to_object_excp_inq(dataTrain))
dataValiup=upsample(datapre.Cate_to_object_excp_inq(dataVali))
dataTest=datapre.Cate_to_object_excp_inq(dataTest)

In [62]:
parameters=[{'max_depth':list(range(5,15)),
            'criterion':["gini", "entropy"],
            'min_samples_split':list(range(200,801,100)),
            'bootstrap':[True],
            'oob_score':[True,False],
            'warm_start':[True,False],
            'max_samples':[1000,2500,3000,3500],
            'min_samples_leaf':[10,40,50,100]},
            {'max_depth':list(range(5,15)),
            'criterion':["gini", "entropy"],
            'min_samples_split':list(range(200,801,100)),
            'bootstrap':[False],
            'warm_start':[True,False],
            'max_samples':[1000,2500,3000,3500],
            'min_samples_leaf':[10,40,50,100]}]


In [396]:
clfemb=rf.best_Random_forest(dataTrainup, dataValiup, parameters, criteria='accuracy', randomstate=0)

In [397]:
clfemb.best_params_

RandomForestClassifier(bootstrap=False, criterion='entropy', max_depth=12,
                       max_samples=2500, min_samples_leaf=50,
                       min_samples_split=500, random_state=0, warm_start=True)

In [121]:
frames=pd.concat([dataTrainup, dataValiup])
datafinaltrain=frames.set_index([pd.Index(range(len(frames)))])

In [122]:
rffinalemb=rf.Random_forest(datafinaltrain, nestimator=100, criter="entropy", mdepth=8, misamples_split=500, 
    misamples_leaf=10, bootstra=False, oobscore=False, warmstart=True, maxsamples=1000, randomstate=0)

In [123]:
dataTestdu=pd.get_dummies(dataTest)
datatestx=np.array(dataTestdu.drop('Default_ind', axis=1))
datatesty=dataTestdu['Default_ind']

In [124]:
confusion_matrix(np.array(datatesty),np.array(rffinalemb.predict(datatestx)))

array([[3666,  933],
       [  94,  307]], dtype=int64)

In [125]:
rffinalemb.score(datatestx,datatesty)

0.7946

RF

In [126]:
pathTrain="C:/Users/12783/Downloads/Data_impute/rf.csv" ##training
pathVali="C:/Users/12783/Downloads/Data_impute/rf_Vali.csv" ## testing
pathTest="C:/Users/12783/Downloads/Data_impute/rf_Test.csv" ## testing

In [127]:
dataTrain=pd.read_csv(pathTrain)
dataVali=pd.read_csv(pathVali)
dataTest=pd.read_csv(pathTest)

In [128]:
dataTrainup=upsample(datapre.Cate_to_object_excp_inq(dataTrain))
dataValiup=upsample(datapre.Cate_to_object_excp_inq(dataVali))
dataTest=datapre.Cate_to_object_excp_inq(dataTest)

In [129]:
parameters=[{'max_depth':list(range(5,15)),
            'criterion':["gini", "entropy"],
            'min_samples_split':list(range(200,801,100)),
            'bootstrap':[True],
            'oob_score':[True,False],
            'warm_start':[True,False],
            'max_samples':[1000,2500,3000,3500],
            'min_samples_leaf':[40,50,100]},
            {'max_depth':list(range(5,15)),
            'criterion':["gini", "entropy"],
            'min_samples_split':list(range(200,801,100)),
            'bootstrap':[False],
            'warm_start':[True,False],
            'max_samples':[1000,2500,3000,3500],
            'min_samples_leaf':[40,50,100]}]

In [396]:
clfrf=rf.best_Random_forest(dataTrainup, dataValiup, parameters, criteria='accuracy', randomstate=0)

In [397]:
clfrf.best_params_

RandomForestClassifier(bootstrap=False, criterion='entropy', max_depth=12,
                       max_samples=2500, min_samples_leaf=50,
                       min_samples_split=500, random_state=0, warm_start=True)

In [130]:
frames=pd.concat([dataTrainup, dataValiup])
datafinaltrain=frames.set_index([pd.Index(range(len(frames)))])

In [135]:
rffinalrf=rf.Random_forest(datafinaltrain, nestimator=100, criter="entropy", mdepth=11, misamples_split=700, 
    misamples_leaf=50, bootstra=False, oobscore=False, warmstart=True, maxsamples=200, randomstate=0)

In [136]:
dataTestdu=pd.get_dummies(dataTest)
datatestx=np.array(dataTestdu.drop('Default_ind', axis=1))
datatesty=dataTestdu['Default_ind']

In [137]:
confusion_matrix(np.array(datatesty),np.array(rffinalrf.predict(datatestx)))

array([[3715,  884],
       [  91,  310]], dtype=int64)

In [138]:
rffinalrf.score(datatestx,datatesty)

0.805

EM

In [141]:
pathTrain="C:/Users/12783/Downloads/Data_impute/Simulated_Data_Train_em.csv" ##training
pathVali="C:/Users/12783/Downloads/Data_impute/Simulated_Data_validation_em.csv" ## testing
pathTest="C:/Users/12783/Downloads/Data_impute/Simulated_Data_Test_em.csv" ## testing

In [142]:
dataTrain=pd.read_csv(pathTrain)
dataVali=pd.read_csv(pathVali)
dataTest=pd.read_csv(pathTest)

In [143]:
dataTrainup=upsample(datapre.Cate_to_object_excp_inq(dataTrain))
dataValiup=upsample(datapre.Cate_to_object_excp_inq(dataVali))
dataTest=datapre.Cate_to_object_excp_inq(dataTest)

In [129]:
parameters=[{'max_depth':list(range(9,20)),
            'criterion':["gini", "entropy"],
            'min_samples_split':[10,20]+list(range(200,801,100)),
            'bootstrap':[True],
            'oob_score':[True,False],
            'warm_start':[True,False],
            'max_samples':[1000,2500,3000,3500],
            'min_samples_leaf':[2,4,6,10,40,50,100]},
            {'max_depth':list(range(9,20)),
            'criterion':["gini", "entropy"],
            'min_samples_split':[10,20]+list(range(200,801,100)),
            'bootstrap':[False],
            'warm_start':[True,False],
            'max_samples':[1000,2500,3000,3500],
            'min_samples_leaf':[2,4,6,10,40,50,100]}]

In [396]:
clfem=rf.best_Random_forest(dataTrainup, dataValiup, parameters, criteria='accuracy', randomstate=0)

In [397]:
clfem.best_params_

RandomForestClassifier(bootstrap=False, criterion='entropy', max_depth=12,
                       max_samples=2500, min_samples_leaf=50,
                       min_samples_split=500, random_state=0, warm_start=True)

In [144]:
frames=pd.concat([dataTrainup, dataValiup])
datafinaltrain=frames.set_index([pd.Index(range(len(frames)))])

In [146]:
rffinalem=rf.Random_forest(datafinaltrain, nestimator=100, criter="gini", mdepth=17, misamples_split=20, 
                           misamples_leaf=6, bootstra=False, oobscore=False, warmstart=True, maxsamples=1000, randomstate=0)

In [147]:
dataTestdu=pd.get_dummies(dataTest)
datatestx=np.array(dataTestdu.drop('Default_ind', axis=1))
datatesty=dataTestdu['Default_ind']

In [148]:
confusion_matrix(np.array(datatesty),np.array(rffinalem.predict(datatestx)))

array([[4521,   78],
       [ 254,  147]], dtype=int64)

In [149]:
rffinalem.score(datatestx,datatesty)

0.9336